In [ ]:
%matplotlib widget
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss

import acr.subjects as subs
import acr
import plotly.express as px

# Scoring Parameters/Times

In [ ]:
info = subs.a13_info
info['paths']['laser1']

# load a second of the data to get the metadata
data = tdt.read_block(info["paths"]['laser1'], t1=0, t2=1)

# and load all of the epocs
ep = tdt.read_block(info["paths"]['laser1'], t1=0, t2=0, evtype=['epocs'])

#start of the file
start_time = np.datetime64(data.info.start_date)

# First times we want to know are the start and end times of the laser stimulation:
stim_on = ep.epocs.Wdr_.onset[2]
stim_off = ep.epocs.Wdr_.offset[2]

# Then we will give ourselves a 5-min buffer on either side of the laser stimulation
stim_on = stim_on - 5*60
stim_off = stim_off + 5*60

# Then we also want to set the times for the second bout of receovery sleep:
# First we will specify RS2 as starting at 2:05pm, which is ~a 5min buffer from when I noted it as ending
rs_start = np.datetime64('2022-09-09T14:05:00')
# Then we get that as a TDT-time
rs_start_sec = (rs_start - start_time) / np.timedelta64(1, 's')

In [ ]:
# we need hypnogram start times (hst) to get the correct times loaded for the hypnogram
# First specify them in seconds:
hst = {}
hst['laser1-stim'] = 16067
hst['laser1-rs'] = 25127
hst['laser1-bl'] = 0

# Then convert to datetime64:
#stim-1
t = tdt.read_block(info["paths"]['laser1'], t1=16067, t2=16068)
hst['laser1-stim'] = np.datetime64(t.info.start_date) + pd.to_timedelta(16067, unit='s')

# RS
rs = tdt.read_block(info["paths"]['laser1'], t1=rs_start_sec, t2=rs_start_sec+1)
hst['laser1-rs'] = np.datetime64(rs.info.start_date) + pd.to_timedelta(rs_start_sec, unit='s')

# BL
bl = tdt.read_block(info["paths"]['laser1-bl'], t1=0, t2=1)
hst['laser1-bl'] = np.datetime64(bl.info.start_date)

In [35]:
# We will have to do some manual loading of the hypnograms because they are discontinuous
def hypno_path(subject, key):
    hypnograms_yaml_file = (
        "/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/acr-hypno-paths.yaml"
    )
    with open(hypnograms_yaml_file) as fp:
        yaml_data = yaml.safe_load(fp)
    root = Path(yaml_data[subject]["hypno-root"])
    path = root / (key + ".txt")
    return path

hypno = {}
p = hypno_path('ACR_13', 'hypno_laser1-stim')
hypno['laser1-stim'] = kh.load_hypno_file(p, hst['laser1-stim'])

p = hypno_path('ACR_13', 'hypno_laser1-rs_chunk1')
hypno['laser1-rs'] = kh.load_hypno_file(p, hst['laser1-rs'])

hypno['laser1-bl'] = acr.io._load_hypno('ACR_13', 'laser1-bl', hst['laser1-bl'])

hypno['laser1'] = pd.concat([hypno['laser1-stim'], hypno['laser1-rs']])

In [36]:
a13_info = subs.a13_info
data, spg, bp, hyp = acr.io.acr_load_master(a13_info, type="xarray", stores=["EEGr", "LFP_", "LFPo"], hyp=False)

read from t=0s to t=15274.98s
Using 1017.2526 Hz as SEV sampling rate for EEGr
read from t=0s to t=15274.98s
Using 610.3516 Hz as SEV sampling rate for LFP_
read from t=0s to t=15274.98s
Using 610.3516 Hz as SEV sampling rate for LFPo
read from t=16067s to t=30020.83s
Using 1017.2526 Hz as SEV sampling rate for EEGr
read from t=16067s to t=30020.83s
Using 610.3516 Hz as SEV sampling rate for LFP_
read from t=16067s to t=30020.83s
Using 610.3516 Hz as SEV sampling rate for LFPo


AttributeError: 'bool' object has no attribute 'keys'

In [ ]:
acr.io.save_dataframes(hyp, a13_info, "-hypno")
acr.io.save_dataframes(bp, a13_info, "-bp")
hyp = acr.io.load_saved_dataset(a13_info, None, '-hypno')

In [ ]:
bp = acr.io.load_saved_dataset(a13_info, ['-LFP_', '-LFPo'], '-bp')

In [ ]:
# get the average delta value for each channel
bp = kpd.pd_utils.add_states_to_dataset(bp, hyp)

In [ ]:
pm = bp['laser1-bl-LFP_'].filt_state(['NREM']).groupby('channel').mean()

In [ ]:
om = bp['laser1-bl-LFPo'].filt_state(['NREM']).groupby('channel').mean()

In [ ]:
p = bp['laser1-LFP_']
o = bp['laser1-LFPo']

In [ ]:
stim_p = p.ts(slice(np.datetime64('2022-09-09T10:05:00'), np.datetime64('2022-09-09T12:10:00')))

In [ ]:
stim_o = o.ts(slice(np.datetime64('2022-09-09T10:05:00'), np.datetime64('2022-09-09T12:10:00')))

In [ ]:
stim_p.filt_state()

In [ ]:
bp['laser1-bl-LFPo'].filt_state(['NREM']).ch(4).delta.median()

In [ ]:
bp['laser1-LFPo'].filt_state(['NREM']).ch(4).delta.median()

In [ ]:
p.reset_index(inplace=True)

In [ ]:
op.groupby(['channel', ).mean().plot()

In [ ]:
stim_p.set_index(['datetime','channel'], inplace=True)

In [ ]:
stim_o.set_index(['datetime','channel'], inplace=True)

In [ ]:
stim_p['delta'] = stim_p['delta']/pm['delta']

In [ ]:
stim_o['delta'] = stim_o['delta']/om['delta']

In [ ]:
op_stim = pd.concat([stim_p, stim_o])

In [ ]:
op_stim.filt_state()

In [ ]:
px.box(op_stim.ri().filt_state(), x='channel', y='delta', color='dtype')

In [ ]:
bp['laser1-LFP_']['dtype'] = 'Probe'
bp['laser1-LFPo']['dtype'] = 'Optrode'

In [ ]:
probes = [bp['laser1-LFPo'], bp['laser1-LFP_']]
ps = pd.concat(probes)